In [ ]:
import sys

sys.path.insert(0, 'backend_functions')

import selection_functions as sf

import importlib

import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb


import awkward
import matplotlib.pyplot as plt
import pandas as pd

import ROOT

import top 
from top import *

import uncertainty_functions 
from uncertainty_functions import *

import xsec_functions 
from xsec_functions import smear_matrix

from ROOT import TH1D, TH2D, TDirectory, TH1F, TH2F

from selection_functions import *

import json

In [ ]:
import io
import json
import os
from datetime import date

In [ ]:
xvar = 'shr_energy_cali'
true_var = 'elec_e'
bins = [0.02, 0.22, 0.42, 0.62, 0.82 , 1.22,  7]

xlow = bins[0]
xhigh = 2.5

x_label = "Electron Energy [GeV]"

x_ticks = [0.02, 0.22, 0.42, 0.62, 0.82 , 1.22]


In [ ]:
if xvar=='tksh_angle': 
    bincenters = 0.5*(np.array(x_ticks)[1:]+np.array(x_ticks)[:-1])

else: 
    bincenters = 0.5*(np.array(x_ticks+[xhigh])[1:]+np.array(x_ticks+[xhigh])[:-1])

In [ ]:
with open('unfolding/variations/FHCVariations_'+xvar+'_July15.json') as f_fhc:
    fhc_dict = json.load(f_fhc)

In [ ]:
with open('unfolding/variations/RHCVariations_'+xvar+'_July15.json') as f_rhc:
    rhc_dict = json.load(f_rhc)

In [ ]:
binwidth = []

for x in range(len(bincenters)): 
    binwidth.append(round(abs((x_ticks+[xhigh])[x+1]-(x_ticks+[xhigh])[x])/2, 2))  
    
binwidth

## PPFX

In [ ]:
fig = plt.figure(figsize=(8, 5)) 

for v in fhc_dict['ppfx']: 
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=fhc_dict['evt_rate'],
        histtype='step', color='black', linewidth=2)    

plt.xlim(xlow, xhigh)
plt.title("PPFX VARIATIONS (FHC)", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)

plt.show()

In [ ]:

fig = plt.figure(figsize=(8, 5)) 

for v in rhc_dict['ppfx']: 
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=rhc_dict['evt_rate'],
        histtype='step', color='black', linewidth=2) 
    
plt.xlim(xlow, xhigh)
plt.title("PPFX VARIATIONS (RHC)", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)
plt.show()

In [ ]:
ppfx_variations = []

fig = plt.figure(figsize=(8, 5)) 

n = 0

for v_fhc, v_rhc in zip(fhc_dict['ppfx'], rhc_dict['ppfx']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    ppfx_variations.append(comb_v)
    
    if n==0: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1, label='UV')
    else: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1)
        
    
    n+=1

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2, label='CV') 
    
plt.xlim(xlow, xhigh)
plt.title("FHC+RHC PPFX Variations", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.legend(fontsize=14, frameon=False)

plt.xlabel('Reconstructed '+x_label, fontsize=14)
plt.ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/ppfx_"+xvar+".pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
fhc_ppfx_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['ppfx'], plot=False, 
                    save=False, xticks=x_ticks, xhigh=xhigh) 

In [ ]:
rhc_ppfx_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['ppfx'], plot=False,  save=False, xticks=x_ticks, xhigh=xhigh) 

In [ ]:
## calculate covariance 

ppfx_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    ppfx_variations, plot=True, xticks=x_ticks, xhigh=xhigh, 
                    axis_label='Reconstructed '+x_label, pot='$7.0 \\times 10^{20}$ POT') 
                    #save="/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/ppfx_") 

In [ ]:
bincenters

In [ ]:
fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_ppfx_dict['fractional_uncertainty'], linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_ppfx_dict['fractional_uncertainty'], linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=ppfx_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.3)

plt.legend(fontsize=13, frameon=False, ncol=2)
plt.title("PPFX Uncertainty", fontsize=16)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/ppfx_"+xvar+"_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

## Beamline Geometry 

In [ ]:
beamline_runs = ['HornCurrent', 'xHorn1', 'yHorn1', 'BeamSpotSize', 'xHorn2', 'yHorn2', 'WaterOnHorns', 
                'xBeamShift', 'yBeamShift', 'zTargetPosition']

In [ ]:
fhc_beamline_unisim_cov = {}
rhc_beamline_unisim_cov = {}
beamline_unisim_cov = {}

i = 0

for v_fhc, v_rhc in zip(fhc_dict['beamline'], rhc_dict['beamline']): 
    
    print(beamline_runs[i])
    
    variations = []
    
    comb_v = [a+b for a,b in zip(v_fhc[0],v_rhc[0])]
    comb_v2 = [a+b for a,b in zip(v_fhc[1],v_rhc[1])]
    
    variations.append(comb_v)
    variations.append(comb_v2)
    
    fig = plt.figure(figsize=(7, 7)) 
    
    gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])

    ax1 = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])
    
    ax1.tick_params(axis = 'both', which = 'major', labelsize = 13)
    ax2.tick_params(axis = 'both', which = 'major', labelsize = 13)
    
    #ax1.errorbar(bincenters, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
    #            color='black', label="CV", xerr=binwidth, linestyle='None')

    ax1.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v, linewidth=1.3, 
        histtype='step', color='cornflowerblue', label='UV (+$1\sigma$)')
    
    ax1.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v2, linewidth=1.3, 
            histtype='step', color='darkgoldenrod', label='UV (-$1\sigma$)')
    
    # Ratios to CV 
    ax2.hist(bincenters, bins,
             weights=[x/y for x,y in zip(comb_v, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])])], 
             color="cornflowerblue", histtype='step', linewidth=1.3)
    
    ax2.hist(bincenters, bins,
             weights=[x/y for x,y in zip(comb_v2, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])])], 
             color="darkgoldenrod", histtype='step', linewidth=1.3)
    
    ax1.set_xlim(xlow, xhigh)
    ax1.set_title("FHC+RHC "+beamline_runs[i]+" Variations", fontsize=15)
    ax1.set_xticks(x_ticks)
    
    ax1.set_ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)
    ax1.legend(fontsize=13)
    
    ax2.yaxis.grid(linestyle="--", color='black', alpha=0.2)
    ax2.xaxis.grid(linestyle="--", color='black', alpha=0.2)

    ax2.set_xlabel('Reconstructed '+x_label, fontsize=14)
    ax2.set_ylabel('UV / CV', fontsize=14)
    ax2.set_xlim(xlow, xhigh)
    ax2.set_ylim(0.95, 1.05)
    ax2.set_xticks(x_ticks)
    
    #plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/beamline_"+beamline_runs[i]+"_"+xvar+".pdf", 
    #        transparent=True, bbox_inches='tight')
    
    plt.show()
    
    # compute unisim covariance
    
    fhc_beamline_unisim_cov[beamline_runs[i]] = calcCov(xvar, bins, 
                                                    fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                                                    v_fhc, plot=False)
    
    rhc_beamline_unisim_cov[beamline_runs[i]] = calcCov(xvar, bins, 
                                                    rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                                                    v_rhc, plot=False)
    
    beamline_unisim_cov[beamline_runs[i]] = calcCov(xvar, bins, 
                                                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                                    variations, plot=False)
    

    i += 1

In [ ]:
for k,d in enumerate([fhc_beamline_unisim_cov, rhc_beamline_unisim_cov, beamline_unisim_cov]): 
    
    cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    
    for variation in d.keys(): 

        for i in range(len(bins)-1): 
            for j in range(len(bins)-1):

                cov[i][j] += d[variation]['cov'][i][j]
                frac_cov[i][j] += d[variation]['frac_cov'][i][j] 


    for i in range(len(bins)-1): 
        for j in range(len(bins)-1):

            if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                    cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
        
    if k==0: 
        
        fhc_beamline_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 
        
        
    elif k==1: 
        rhc_beamline_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 
        
    elif k==2: 
        beamline_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 


In [ ]:
fig = plt.figure(figsize=(10, 6))

plt.pcolor(bins, bins, beamline_unisim_dict['cov'], cmap='OrRd', edgecolors='k')
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
     
plt.xlabel("Reconstructed "+x_label, fontsize=15)
plt.ylabel("Reconstructed "+x_label, fontsize=15)

cbar.set_label(label='$\\nu^{2}$ / $7.0 \\times 10^{20}$ POT$^{2}$', fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)

plt.title('Covariance Matrix', fontsize=16)
#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/beamline_"+xvar+"_cov.pdf", 
#            transparent=True, bbox_inches='tight')
plt.close()

In [ ]:
fig = plt.figure(figsize=(10, 6))

plt.pcolor(bins, bins, beamline_unisim_dict['frac_cov'], cmap='OrRd', edgecolors='k', vmin=-0.001, vmax=0.01)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
     
plt.xlabel("Reconstructed "+x_label, fontsize=15)
plt.ylabel("Reconstructed "+x_label, fontsize=15)

cbar.set_label(label='$\\nu^{2}$ / $7.0 \\times 10^{20}$ POT$^{2}$', fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)

plt.title('Fractional Covariance Matrix', fontsize=16)
#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/beamline_"+xvar+"_frac_cov.pdf", 
#            transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 6))

plt.pcolor(bins, bins, beamline_unisim_dict['cor'], cmap='OrRd', edgecolors='k', vmin=-0.5, vmax=1)
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
     
plt.xlabel("Reconstructed "+x_label, fontsize=15)
plt.ylabel("Reconstructed "+x_label, fontsize=15)

cbar.set_label(label='$\\nu^{2}$ / $7.0 \\times 10^{20}$ POT$^{2}$', fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)

plt.title('Correlation Matrix', fontsize=16)
#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/beamline_"+xvar+"_cor.pdf", 
#            transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=beamline_unisim_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_beamline_unisim_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_beamline_unisim_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.15)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("Beamline Uncertainty", fontsize=16)

#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/beamline_"+xvar+"_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()

## GENIE ms

In [ ]:
genie_variations = []

fig = plt.figure(figsize=(8, 5)) 

n = 0

for v_fhc, v_rhc in zip(fhc_dict['genie_ms'], rhc_dict['genie_ms']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    genie_variations.append(comb_v)
    
    if n==0: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1, label='UV')
    else: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1)
        
    n+=1

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2, label='CV') 
    
plt.xlim(xlow, xhigh)
plt.title("FHC+RHC GENIE (ms) Variations", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.legend(fontsize=14, frameon=False)

plt.xlabel('Reconstructed '+x_label, fontsize=14)
plt.ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_"+xvar+".pdf", 
            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
fhc_genie_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['genie_ms'], plot=False) 

rhc_genie_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['genie_ms'], plot=False) 

In [ ]:
genie_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                     [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                     genie_variations, xticks=x_ticks, xhigh=xhigh,
                     axis_label='Reconstructed '+x_label, pot='$7.0 \\times 10^{20}$ POT',
                     plot=True, save="/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_")


In [ ]:
fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=genie_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_genie_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_genie_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.35)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("GENIE (ms) Uncertainty", fontsize=16)


plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_"+xvar+"_frac.pdf", 
            transparent=True, bbox_inches='tight')

plt.show()

## GENIE us

In [ ]:
genie_unisims = ['RPA', 'RPA', 'CCMEC', 'AxFFCCQE', 'VecFFCCQE', 'DecayAngMEC', 'ThetaDelta2Npi', 'ThetaDelta2NRad', 
                          'NormCCCOH', 'NormNCCOH', 'xsr_scc_Fv3', 'xsr_scc_Fa3']

In [ ]:
genie_unisim_cov = {}
fhc_genie_unisim_cov = {}
rhc_genie_unisim_cov = {}


i = 1

for v_fhc, v_rhc in zip(fhc_dict['genie_us'], rhc_dict['genie_us']): 
    
    print(genie_unisims[i])
    
    variations = []
    
    comb_v = [a+b for a,b in zip(v_fhc[0],v_rhc[0])]
    
    if i==1:
        comb_v2 = [a+b for a,b in zip(v_fhc[1],v_rhc[1])]
        variations.append(comb_v)
        variations.append(comb_v2)
    
    else: 
        variations.append(comb_v)
    
    fig = plt.figure(figsize=(7, 7)) 
    
    gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])

    ax1 = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])
    
    ax1.tick_params(axis = 'x', which = 'major', labelsize = 13)
    ax2.tick_params(axis = 'x', which = 'major', labelsize = 13)
    
    ax1.tick_params(axis = 'y', which = 'major', labelsize = 13)
    ax2.tick_params(axis = 'y', which = 'major', labelsize = 13)
    
    #ax1.errorbar(bincenters, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
    #            color='black', label="CV", xerr=binwidth, linestyle='None')
    
    if i==1: 
        
        ax1.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v, linewidth=1.3, 
        histtype='step', color='cornflowerblue', label='UV (+1$\\sigma$)')
        
        ax1.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v2, linewidth=1.3, 
        histtype='step', color='darkgoldenrod', label='UV (-1$\\sigma$)')
        
        ax1.legend(fontsize=13)
        
    else: 
        ax1.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v, linewidth=1.3, 
        histtype='step', color='cornflowerblue')
    
    # Ratios to CV 
    ax2.hist(bincenters, bins,
             weights=[x/y for x,y in zip(comb_v, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])])], 
             color="cornflowerblue", histtype='step', linewidth=1.3)
    
    #print([x/y for x,y in zip(comb_v, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])])])
    
    if i==1: 
        ax2.hist(bincenters, bins,
             weights=[x/y for x,y in zip(comb_v2, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])])], 
             color="darkgoldenrod", histtype='step', linewidth=1.3)
    
    
    ax1.set_xlim(xlow, xhigh)
    if i==1: 
        ax1.set_title("FHC+RHC "+genie_unisims[i]+" Variations", fontsize=15)
    else: 
        ax1.set_title("FHC+RHC "+genie_unisims[i]+" Variation", fontsize=15)
    ax1.set_xticks(x_ticks)
    
    ax1.set_ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)
    
    ax2.yaxis.grid(linestyle="--", color='black', alpha=0.2)
    ax2.xaxis.grid(linestyle="--", color='black', alpha=0.2)

    ax2.set_xlabel('Reconstructed '+x_label, fontsize=14)
    ax2.set_ylabel('UV / CV', fontsize=14)
    ax2.set_xlim(xlow, xhigh)
    
    if (genie_unisims[i] == 'CCMEC') or (genie_unisims[i] =='ThetaDelta2NRad') or ('xsr' in genie_unisims[i]): 
        ax2.set_ylim(0.99, 1.01)
    else: 
        ax2.set_ylim(0.95, 1.05)
    
    ax2.set_xticks(x_ticks)
    
    plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_us_"+genie_unisims[i]+"_"+xvar+".pdf", 
            transparent=True, bbox_inches='tight')
    
    plt.show()
    
    fhc_genie_unisim_cov[genie_unisims[i]] = calcCov(xvar, bins, 
                                                    fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                                                    v_fhc, plot=False, save=False)
    
    rhc_genie_unisim_cov[genie_unisims[i]] = calcCov(xvar, bins, 
                                                    rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                                                    v_rhc, plot=False, save=False)
    
    genie_unisim_cov[genie_unisims[i]] = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                     [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                                     variations, plot=False, save=False)
    

    i += 1

In [ ]:
for k,d in enumerate([fhc_genie_unisim_cov, rhc_genie_unisim_cov, genie_unisim_cov]): 
    
    cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    cor = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
    
    for variation in d.keys(): 

        for i in range(len(bins)-1): 
            for j in range(len(bins)-1):

                cov[i][j] += d[variation]['cov'][i][j]
                frac_cov[i][j] += d[variation]['frac_cov'][i][j] 


    for i in range(len(bins)-1): 
        for j in range(len(bins)-1):

            if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
                    cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))
        
    if k==0: 
        fhc_genie_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 
        
        
    elif k==1: 
        rhc_genie_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 
        
    elif k==2: 
        genie_unisim_dict = {
            'cov' : cov, 
            'frac_cov' : frac_cov,
            'cor' : cor,
            'fractional_uncertainty' : np.sqrt(np.diag(frac_cov))
        } 


In [ ]:
# covariance

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, genie_unisim_dict['cov'], cmap='OrRd', edgecolors='k')
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)
        
plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xlabel('Reconstructed '+x_label, fontsize=15)
plt.ylabel('Reconstructed '+x_label, fontsize=15)

plt.title('Covariance Matrix', fontsize=16)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_us_"+xvar+"_cov.pdf", 
            transparent=True, bbox_inches='tight')


plt.show()

In [ ]:
# frac covariance

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, genie_unisim_dict['frac_cov'], cmap='OrRd', edgecolors='k', vmin=-0.001, vmax=0.003)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)
        
plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xlabel('Reconstructed '+x_label, fontsize=15)
plt.ylabel('Reconstructed '+x_label, fontsize=15)

plt.title('Fractional Covariance Matrix', fontsize=16)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_us_"+xvar+"_frac_cov.pdf", 
            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
# correlation

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins, bins, genie_unisim_dict['cor'], cmap='OrRd', edgecolors='k', vmin=-1, vmax=1)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(x_ticks, fontsize=13)
plt.yticks(x_ticks,fontsize=13)
        
plt.xlim(bins[0], xhigh)
plt.ylim(bins[0], xhigh)

plt.xlabel('Reconstructed '+x_label, fontsize=15)
plt.ylabel('Reconstructed '+x_label, fontsize=15)

plt.title('Correlation Matrix', fontsize=16)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_us_"+xvar+"_cor.pdf", 
            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:


fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=genie_unisim_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_genie_unisim_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_genie_unisim_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.065)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("GENIE (us) Uncertainty", fontsize=16)


plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/genie_us_"+xvar+"_frac.pdf", 
            transparent=True, bbox_inches='tight')

plt.show()

## GEANT4

In [ ]:
geant4_variations = []

fig = plt.figure(figsize=(8, 5)) 

n = 0

for v_fhc, v_rhc in zip(fhc_dict['geant4'], rhc_dict['geant4']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    geant4_variations.append(comb_v)
    
    if n==0: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1, label='UV')
    else: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1)
        
    
    n+=1

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2, label='CV') 
    
plt.xlim(xlow, xhigh)
plt.title("FHC+RHC GEANT4 Variations", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.legend(fontsize=14, frameon=False)

plt.xlabel('Reconstructed '+x_label, fontsize=14)
plt.ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/geant4_"+xvar+".pdf", 
            transparent=True, bbox_inches='tight')

plt.show()


In [ ]:
fhc_geant4_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['geant4'], plot=False) 

rhc_geant4_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['geant4'], plot=False) 

In [ ]:
geant4_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    geant4_variations, plot=True, xticks=x_ticks, xhigh=xhigh, axis_label="Reconstructed "+x_label, 
                     pot='$7.0 \\times 10^{20}$ POT', 
                    save="/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/geant4_") 

In [ ]:


fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=geant4_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_geant4_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_geant4_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.05)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("GEANT4 Uncertainty", fontsize=16)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/geant4_"+xvar+"_frac.pdf", 
            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
fhc_geant4_dict['fractional_uncertainty']

## Detector systematics -- Using Reco X

In [ ]:
with open('unfolding/variations/FHCDetSysVariations_July31.json') as f_fhc_detsys:
    fhc_detsys_dict = json.load(f_fhc_detsys)

In [ ]:
fhc_detsys_dict['LYAttenuation_intrinsic'] = [0 for i in range(len(fhc_detsys_dict['CV_intrinsic'] ))]

In [ ]:
with open('unfolding/variations/RHCDetSysVariations_July31.json') as f_rhc_detsys:
    rhc_detsys_dict = json.load(f_rhc_detsys)

In [ ]:
xvar_detsys = "reco_nu_vtx_sce_x"
bins_detsys = [0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]

In [ ]:
bincenters_detsys = 0.5*(np.array(bins_detsys)[1:]+np.array(bins_detsys)[:-1])

In [ ]:
x_err_detsys = [ round(abs(bins_detsys[x+1]-bins_detsys[x])/2, 3) for x in range(len(bins_detsys)-1) ]

In [ ]:
for variation in intrinsic_detvar_run3_rhc.keys(): 
    
    if variation=='CV_intrinsic': 
        continue
        
    if variation=='LYAttenuation_intrinsic': 
        comb_v = rhc_detsys_dict[variation]
        comb_cv = rhc_detsys_dict['CV_intrinsic']
        
    else: 
        comb_v = [a+b for a,b in zip(fhc_detsys_dict[variation], rhc_detsys_dict[variation])]
        comb_cv = [a+b for a,b in zip(fhc_detsys_dict['CV_intrinsic'], rhc_detsys_dict['CV_intrinsic'])]
        
    fig = plt.figure(figsize=(7, 7)) 
    
    gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1])

    ax1 = plt.subplot(gs[0])
    ax2 = plt.subplot(gs[1])
    
    ax1.tick_params(axis = 'x', which = 'major', labelsize = 13)
    ax2.tick_params(axis = 'x', which = 'major', labelsize = 13)
    
    ax1.tick_params(axis = 'y', which = 'major', labelsize = 13)
    ax2.tick_params(axis = 'y', which = 'major', labelsize = 13)
        
    ax1.hist(bincenters_detsys, bins_detsys, histtype='step', range=[bins_detsys[0], bins_detsys[-1]], 
                 weights=comb_v, color='cornflowerblue', linewidth=1.3)
    # Ratios to CV 
    
    ratio = [x/y for x,y in zip(comb_v, comb_cv)]
    
    ax2.hist(bincenters_detsys, bins_detsys,
             weights=ratio, 
             color="cornflowerblue", histtype='step', linewidth=1.3)
    
    if variation=='LYAttenuation_intrinsic': 
        ax1.set_title(variation[:-10]+'\nIntrinsic $\\nu_{e}$ CC events (RHC Only)', fontsize=15)
        
    else: 
        ax1.set_title(variation[:-10]+'\nIntrinsic $\\nu_{e}$ CC events', fontsize=15)
    ax1.set_ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)
    ax1.set_xlim(0, 250)
    ax1.set_ylim(0, 25)
    
    
    ax2.set_xlabel("Reconstructed "+x_label, fontsize=14)
    ax2.set_ylabel('UV / CV', fontsize=14)
    ax2.yaxis.grid(linestyle="--", color='black', alpha=0.2)
    ax2.xaxis.grid(linestyle="--", color='black', alpha=0.2)
    ax2.set_xlim(0, 250)
    ax2.set_ylim(0.9, 1.15)
    
    plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/detsys_"+variation+"_recox.pdf", 
            transparent=True, bbox_inches='tight')
    
    plt.show()

In [ ]:
fhc_detsys_dict.keys()

In [ ]:
xvar

In [ ]:
detsys_cov_nueCC = {}

for variation in intrinsic_detvar_run3_rhc.keys(): 
    
    print(variation)
    
    if variation=='CV': 
        continue
        
    if variation=='LYAttenuation_intrinsic': 
        
        detsys_cov_nueCC[variation] = calcCov(xvar_detsys, bins_detsys, 
                                          rhc_detsys_dict['CV_intrinsic'], 
                                              rhc_detsys_dict['CV_intrinsic'], 
                                              [rhc_detsys_dict[variation]])
        
    else: 
   
        detsys_cov_nueCC[variation] = calcCov(xvar_detsys, bins_detsys, 
                                          [a+b for a,b in zip(fhc_detsys_dict['CV_intrinsic'], rhc_detsys_dict['CV_intrinsic'])], 
                                              [a+b for a,b in zip(fhc_detsys_dict['CV_intrinsic'], rhc_detsys_dict['CV_intrinsic'])], 
                                              [[a+b for a,b in zip(fhc_detsys_dict[variation], rhc_detsys_dict[variation])]])
  

In [ ]:
cov = [ [0]*(len(bins_detsys)-1) for x in range(len(bins_detsys)-1) ]
frac_cov = [ [0]*(len(bins_detsys)-1) for x in range(len(bins_detsys)-1) ]
cor = [ [0]*(len(bins_detsys)-1) for x in range(len(bins_detsys)-1) ]

for i in range(len(bins_detsys)-1): 
    for j in range(len(bins_detsys)-1):
            
        cov[i][j] = sum([detsys_cov_nueCC[x]['cov'][i][j] for x in intrinsic_detvar_run3_rhc.keys() if x is not 'CV_intrinsic'])
        frac_cov[i][j] = sum([detsys_cov_nueCC[x]['frac_cov'][i][j] for x in intrinsic_detvar_run3_rhc.keys() if x is not 'CV_intrinsic'])
        
# compute the corr matrix 
for i in range(len(cov)): 
    for j in range(len(cov[i])): 

        if np.sqrt(cov[i][i])*np.sqrt(cov[j][j]) != 0: 
            cor[i][j] = cov[i][j] / (np.sqrt(cov[i][i])*np.sqrt(cov[j][j]))

nueCC_detsys_dict = {
    'cov' : cov, 
    'frac_cov' : frac_cov,
    'fractional_uncertainty' : np.sqrt(np.diag(frac_cov)),
    'cor' : cor
} 



In [ ]:
# cov

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins_detsys, bins_detsys, nueCC_detsys_dict['cov'], cmap='OrRd', edgecolors='k')
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)


plt.xlabel("Reconstructed X [cm]", fontsize=14)
plt.ylabel("Reconstructed X [cm]", fontsize=14)

plt.title('Covariance Matrix', fontsize=16)

plt.show()

In [ ]:
# frac cov 

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins_detsys, bins_detsys, nueCC_detsys_dict['frac_cov'], cmap='OrRd', edgecolors='k')
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)


plt.xlabel("Reconstructed X [cm]", fontsize=14)
plt.ylabel("Reconstructed X [cm]", fontsize=14)

plt.title('Fractional Covariance Matrix', fontsize=16)


plt.show()

In [ ]:
# cor

fig = plt.figure(figsize=(10, 6))
        
plt.pcolor(bins_detsys, bins_detsys, nueCC_detsys_dict['cor'], cmap='OrRd', edgecolors='k', vmin=-1, vmax=1)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)
cbar.set_label(label="$\\nu^{2}$ / $7.0\\times10^{20}$  POT$^{2}$", fontsize=15)
        
plt.xticks(fontsize=13)
plt.yticks(fontsize=13)


plt.xlabel("Reconstructed X [cm]", fontsize=14)
plt.ylabel("Reconstructed X [cm]", fontsize=14)

plt.title('Correlation Matrix', fontsize=16)



plt.show()

In [ ]:

x_err = [ round(abs(bins[x+1]-bins[x])/2, 3) for x in range(len(bins)-1) ]

plt.figure(figsize=(8, 5))

#cv_bdtcut = 8767.2+5945.0 # GENIE/PPFX weighted number of events for stat uncertainty 
#cv_bdtcut_err = [np.sqrt(x)/x for x in cv_bdtcut]

plt.hist(bincenters_detsys, bins_detsys, weights=nueCC_detsys_dict['fractional_uncertainty'], histtype='step', 
         label='After BDT Cut (from 14712.2 GENIE/PPFX-tuned CV evts)', color='black')

#plt.errorbar(bincenters_detsys, nueCC_detsys_dict['fractional_uncertainty'], fmt='none', 
#             yerr=cv_bdtcut_err, color='gray')


plt.hlines(np.average(nueCC_detsys_dict['fractional_uncertainty']), 0, 250, linestyle='--', color='darkgray', linewidth=2,
           label='Average = '+str(round(np.average(nueCC_detsys_dict['fractional_uncertainty']), 5)))

plt.xlim(0, 250)
plt.ylim(0, 0.23)
plt.legend(fontsize=13, loc='upper left', frameon=False)

plt.xlabel("Reconstructed X [cm]", fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xticks(fontsize=13)
plt.yticks(fontsize=13)

plt.title("FHC Run 1 & RHC Run 3 Detector Systematics\n$\\nu_{e}$ CC Intrinsic Sample", fontsize=15)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/detsys_recox_frac.pdf", 
            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
avg_detsys_dict = {
    'fractional_uncertainty' : [np.average(nueCC_detsys_dict['fractional_uncertainty']) for x in range(len(bins))], 
}

In [ ]:
frac_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for i in range(len(bins)-1): 
        frac_cov[i][i] = np.average(nueCC_detsys_dict['fractional_uncertainty'])**2

avg_detsys_dict['frac_cov'] = frac_cov

## Stat uncertainty (MC background)

In [ ]:
# doesn't include EXT uncertainty 
# sum of the weights squared

mc_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
mc_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

fhc_mc_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
rhc_mc_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

mc_sumw2 = [a+b for a,b in zip(fhc_dict['mc_bkgd_sumw2'], rhc_dict['mc_bkgd_sumw2'])]
evt_rate = [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])]

fhc_mc_stat_percent_error = []
rhc_mc_stat_percent_error = []

# MC background stat uncertainty
for i in range(len(bins)-1):
    
    mc_stat_cov[i][i] = mc_sumw2[i]
    mc_frac_stat_cov[i][i] = mc_sumw2[i] / (evt_rate[i]**2)
    
    fhc_mc_frac_stat_cov[i][i] = fhc_dict['mc_bkgd_sumw2'][i]/ (fhc_dict['evt_rate'][i]**2)
    rhc_mc_frac_stat_cov[i][i] = rhc_dict['mc_bkgd_sumw2'][i]/ (rhc_dict['evt_rate'][i]**2)
    
    fhc_mc_stat_percent_error.append( np.sqrt( fhc_mc_frac_stat_cov[i][i]) ) #fhc_dict['mc_bkgd_sumw2'][i]/(fhc_dict['evt_rate'][i]**2) ) )
    rhc_mc_stat_percent_error.append(np.sqrt(rhc_mc_frac_stat_cov[i][i])) #rhc_dict['mc_bkgd_sumw2'][i]/(rhc_dict['evt_rate'][i]**2)))

    
mc_stat_percent_error = np.sqrt(np.diag(mc_frac_stat_cov))
mc_stat_percent_error

In [ ]:


fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=mc_stat_percent_error, color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_mc_stat_percent_error, 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_mc_stat_percent_error, 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.18)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("MC Background Statistical Uncertainty", fontsize=16)


#plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/mc_bkgd_stat_"+xvar+"_frac.pdf", 
#            transparent=True, bbox_inches='tight')

plt.show()


## Stat Uncertainty (EXT)

In [ ]:
# EXT uncertainty 
# sum of the weights squared

ext_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
ext_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

fhc_ext_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
rhc_ext_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

ext_sumw2 = [a+b for a,b in zip(fhc_dict['ext_sumw2'], rhc_dict['ext_sumw2'])]

fhc_ext_stat_percent_error = []
rhc_ext_stat_percent_error = []

# MC background stat uncertainty
for i in range(len(bins)-1):
    
    ext_stat_cov[i][i] = ext_sumw2[i]
    ext_frac_stat_cov[i][i] = ext_sumw2[i] / evt_rate[i]**2 
    
    fhc_ext_frac_stat_cov[i][i] = fhc_dict['ext_sumw2'][i]/ (fhc_dict['evt_rate'][i]**2)
    rhc_ext_frac_stat_cov[i][i] = rhc_dict['ext_sumw2'][i]/ (rhc_dict['evt_rate'][i]**2)
    
    fhc_ext_stat_percent_error.append(np.sqrt( fhc_ext_frac_stat_cov[i][i] )) #fhc_dict['ext_sumw2'][i]/fhc_dict['evt_rate'][i]**2))
    rhc_ext_stat_percent_error.append(np.sqrt( rhc_ext_frac_stat_cov[i][i] ))#rhc_dict['ext_sumw2'][i]/rhc_dict['evt_rate'][i]**2))
    
ext_stat_percent_error = np.sqrt(np.diag(ext_frac_stat_cov))
ext_stat_percent_error

In [ ]:
fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=ext_stat_percent_error, color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", color='limegreen',
         weights=fhc_ext_stat_percent_error, 
         linestyle=(0, (1, 1)), linewidth=2)
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_ext_stat_percent_error, 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.18)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("EXT Statistical Uncertainty", fontsize=16)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/ext_stat_"+xvar+"_frac.pdf", 
            transparent=True, bbox_inches='tight')

plt.show()

## Data Stat Uncertainty (estimated BEAM ON)

In [ ]:
# background subtracted event rate (use MC for closure test/fake data studies, beam ON count for real data result)

print("Make sure to update to beam on statistics when using real data !")
    
beamon_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
fhc_beamon_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
rhc_beamon_frac_stat_cov = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

fhc_beamon_stat_percent_error = []
rhc_beamon_stat_percent_error = []
    
for i in range(len(bins)-1): 
    
    beamon_frac_stat_cov[i][i] = evt_rate[i]/(evt_rate[i]**2)
    fhc_beamon_frac_stat_cov[i][i] = fhc_dict['evt_rate'][i]/(fhc_dict['evt_rate'][i]**2)
    rhc_beamon_frac_stat_cov[i][i] = rhc_dict['evt_rate'][i]/(rhc_dict['evt_rate'][i]**2)


fhc_beamon_stat_percent_error = np.sqrt(np.diag(fhc_beamon_frac_stat_cov))
rhc_beamon_stat_percent_error = np.sqrt(np.diag(rhc_beamon_frac_stat_cov))
beamon_stat_percent_error = np.sqrt(np.diag(beamon_frac_stat_cov))

print(beamon_stat_percent_error) 

In [ ]:
fig = plt.figure(figsize=(8, 5))  


plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=beamon_stat_percent_error, color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_beamon_stat_percent_error, 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_beamon_stat_percent_error, 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks[:-1], fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.8)


plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("Beam-On Uncertainty (estimated with MC)", fontsize=16)

plt.show()

## Stat uncertainty (response matrix)

In [ ]:
rmatrix_variations = []

fig = plt.figure(figsize=(8, 5)) 
n = 0

for v_fhc, v_rhc in zip(fhc_dict['response_matrix'], rhc_dict['response_matrix']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    rmatrix_variations.append(comb_v)
    
    if n==0: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1, label="UV")
        
    else: 
        plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
            histtype='step', color='cornflowerblue', linewidth=1)
        
    n+=1

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2, label='CV') 
    
plt.xlim(xlow, 2.5)
plt.title("FHC+RHC Response Matrix Variations", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.legend(fontsize=14, frameon=False)

plt.xlabel("Reconstructed "+x_label, fontsize=14)
plt.ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/rmatrix_"+xvar+".pdf", 
            transparent=True, bbox_inches='tight')

plt.show()

In [ ]:
fhc_rmatrix_dict = calcCov(xvar, bins, fhc_dict['evt_rate'], fhc_dict['evt_rate'], 
                    fhc_dict['response_matrix'], plot=False, save=False) 

rhc_rmatrix_dict = calcCov(xvar, bins, rhc_dict['evt_rate'], rhc_dict['evt_rate'], 
                    rhc_dict['response_matrix'], plot=False, save=False) 

In [ ]:
rmatrix_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    rmatrix_variations, plot=False, save=False) 

In [ ]:
fig = plt.figure(figsize=(8, 5))  

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=rmatrix_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_rmatrix_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_rmatrix_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.12)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("Response Matrix Uncertainty", fontsize=16)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/rmatrix_"+xvar+"_frac.pdf", 
            transparent=True, bbox_inches='tight')

plt.show()

## Dirt Uncertainty 

In [ ]:
dirt_variations = []

fig = plt.figure(figsize=(8, 5)) 


comb_v = [a+b for a,b in zip(fhc_dict['dirt'],rhc_dict['dirt'])]
dirt_variations.append(comb_v)
    
plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1, label='UV')

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['cv_dirt'], rhc_dict['cv_dirt'])],
        histtype='step', color='black', linewidth=2, label='CV') 
    
plt.xlim(xlow, xhigh)
plt.title("FHC+RHC Dirt Variation", fontsize=15)
plt.ylabel('$\\nu$ / $7.0 \\times 10^{20}$ POT' , fontsize=14)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel('Reconstructed '+x_label, fontsize=14)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/dirt_"+xvar+".pdf", 
            transparent=True, bbox_inches='tight')
plt.show()

In [ ]:
fhc_dirt_dict = dirt_unisim(xvar, bins, fhc_dict['evt_rate'], fhc_dict['cv_dirt'], 1.0, isrun3=False, plot=False, 
                               x_label=None, title=None)

rhc_dirt_dict = dirt_unisim(xvar, bins, rhc_dict['evt_rate'], rhc_dict['cv_dirt'], 1.0, isrun3=True, plot=False, 
                               x_label=None, title=None)

In [ ]:
dirt_dict = dirt_unisim(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                        [a+b for a,b in zip(fhc_dict['cv_dirt'], rhc_dict['cv_dirt'])], 
                        1.0, plot=False, x_label=None, title=None)

In [ ]:
fhc_dirt_dict['fractional_uncertainty']

In [ ]:
fig = plt.figure(figsize=(8, 5))  

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Combined", 
         weights=dirt_dict['fractional_uncertainty'], color='black', linewidth=2)

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="FHC", 
         weights=fhc_dirt_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), linewidth=2, color='limegreen')
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="RHC", 
         weights=rhc_dirt_dict['fractional_uncertainty'], 
         linestyle=(0, (1, 1)), color='red', linewidth=2)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reconstructed " + x_label, fontsize=14)
plt.ylabel("Fractional Uncertainty", fontsize=14)

plt.xlim(xlow, xhigh)
plt.ylim(0, 0.025)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("Dirt Uncertainty", fontsize=15)

plt.savefig("/uboone/data/users/kmiller/uBNuMI_CCNp/plots/combined/dirt_"+xvar+"_frac.pdf", 
            transparent=True, bbox_inches='tight')

plt.show()

## POT counting 

In [ ]:
pot_variations = []

fig = plt.figure(figsize=(8, 5)) 

for v_fhc, v_rhc in zip(fhc_dict['pot_counting'], rhc_dict['pot_counting']): 
    
    comb_v = [a+b for a,b in zip(v_fhc,v_rhc)]
    pot_variations.append(comb_v)
    
    plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, weights=comb_v,
        histtype='step', color='cornflowerblue', linewidth=1)

plt.hist(0.5*(np.array(bins)[1:]+np.array(bins)[:-1]), bins, 
        weights=[a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])],
        histtype='step', color='black', linewidth=2) 
    
plt.xlim(xlow, 2.5)
plt.title("FHC+RHC POT Counting Variations", fontsize=15)

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel(x_label, fontsize=14)
plt.show()

In [ ]:
pot_dict = calcCov(xvar, bins, [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    [a+b for a,b in zip(fhc_dict['evt_rate'], rhc_dict['evt_rate'])], 
                    pot_variations, plot=False, save=False) 

In [ ]:
pot_dict['fractional_uncertainty']

## Full Uncertainty 

In [ ]:
fhc_frac_cov_dict = {
    'ppfx' : fhc_ppfx_dict['frac_cov'], 
    'beamline' : fhc_beamline_unisim_dict['frac_cov'], 
    'genie_ms' : fhc_genie_dict['frac_cov'], 
    'genie_us': fhc_genie_unisim_dict['frac_cov'], 
    'geant4' : fhc_geant4_dict['frac_cov'],
    'detector' :  [ [0]*(len(bins)-1) for x in range(len(bins)-1) ], 
    'pot_counting' : [ [round(0.02**2, 2)]*(len(bins)-1) for x in range(len(bins)-1) ], 
    'dirt' : fhc_dirt_dict['frac_cov'],
    'mc_stat' : fhc_mc_frac_stat_cov, # background only 
    'ext_stat' : fhc_ext_frac_stat_cov, 
    'response_matrix' : fhc_rmatrix_dict['frac_cov']
    #'beamon_stat' : fhc_beamon_frac_stat_cov
}



for i in range(len(bins)-1): 
    fhc_frac_cov_dict['detector'][i][i] = parameters(ISRUN3=False)['detsys_flat']**2



In [ ]:
rhc_frac_cov_dict = {
    'ppfx' : rhc_ppfx_dict['frac_cov'], 
    'beamline' : rhc_beamline_unisim_dict['frac_cov'], 
    'genie_ms' : rhc_genie_dict['frac_cov'], 
    'genie_us': rhc_genie_unisim_dict['frac_cov'], 
    'geant4' : rhc_geant4_dict['frac_cov'],
    'detector' :  [ [0]*(len(bins)-1) for x in range(len(bins)-1) ],
    'pot_counting' : [ [round(0.02**2, 2)]*(len(bins)-1) for x in range(len(bins)-1) ],  
    'dirt' : rhc_dirt_dict['frac_cov'],
    'mc_stat' : rhc_mc_frac_stat_cov, # background only 
    'ext_stat' : rhc_ext_frac_stat_cov, 
    'response_matrix' : fhc_rmatrix_dict['frac_cov']
    #'beamon_stat' : rhc_beamon_frac_stat_cov
}

for i in range(len(bins)-1): 
    rhc_frac_cov_dict['detector'][i][i] = parameters(ISRUN3=True)['detsys_flat']**2



In [ ]:
frac_cov_dict = {
    'ppfx' : ppfx_dict['frac_cov'], 
    'beamline' : beamline_unisim_dict['frac_cov'], 
    'genie_ms' : genie_dict['frac_cov'], 
    'genie_us': genie_unisim_dict['frac_cov'], 
    'geant4' : geant4_dict['frac_cov'],
    'detector' : avg_detsys_dict['frac_cov'], 
    'pot_counting' : pot_dict['frac_cov'], 
    'dirt' : dirt_dict['frac_cov'],
    'mc_stat' : mc_frac_stat_cov, # background only 
    'ext_stat' : ext_frac_stat_cov, 
    'response_matrix' : rmatrix_dict['frac_cov']
    #'beamon_stat' : beamon_frac_stat_cov
}

In [ ]:
fhc_tot_frac_cov, fhc_tot_abs_cov = plotFullCov(fhc_frac_cov_dict, xvar, fhc_dict['evt_rate'], 
                                                bins, xlow, xhigh, x_ticks=x_ticks, save=False, 
                                                axis_label='Reco '+x_label)


In [ ]:
np.sqrt(np.diag(fhc_tot_frac_cov))

In [ ]:
rhc_tot_frac_cov, rhc_tot_abs_cov = plotFullCov(rhc_frac_cov_dict, xvar, rhc_dict['evt_rate'], 
                                                bins, xlow, xhigh, x_ticks=x_ticks, save=False, 
                                                axis_label='Reco '+x_label)

In [ ]:
tot_frac_cov, tot_abs_cov = plotFullCov(frac_cov_dict, xvar, evt_rate, bins, xlow, xhigh, x_ticks=x_ticks, save=False, 
                      axis_label='Reco '+x_label)

In [ ]:
# add ppfx & beamline geometry in quadrature
frac_cov_dict['flux'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(frac_cov_dict['ppfx'], frac_cov_dict['beamline'])]

# add genie in quadrature
frac_cov_dict['genie_all'] = [ [x+y for x,y in zip(a,b)] for a,b in zip(frac_cov_dict['genie_ms'], frac_cov_dict['genie_us'])]

In [ ]:
print("MAKE sure to update to include beam on stat!")
frac_cov_dict['stat_all'] = [ [x+y+z for x,y,z in zip(b,c,d)] for b,c,d in zip( frac_cov_dict['response_matrix'], frac_cov_dict['mc_stat'], frac_cov_dict['ext_stat'])]


In [ ]:
# clean away nans
v = np.array(tot_frac_cov)
v[np.isnan(v)] = 0
tot_frac_cov = v

In [ ]:
np.sqrt(np.diagonal(fhc_tot_frac_cov))

In [ ]:
frac_unc_dict = {
    'flux' : np.sqrt(np.diagonal(frac_cov_dict['flux'])), 
    'genie' : np.sqrt(np.diagonal(frac_cov_dict['genie_all'])), 
    'geant4' : np.sqrt(np.diagonal(frac_cov_dict['geant4'])),
    'detector' : np.sqrt(np.diagonal(frac_cov_dict['detector'])), 
    'pot_counting' : np.sqrt(np.diagonal(frac_cov_dict['pot_counting'])), 
    'dirt' : np.sqrt(np.diagonal(frac_cov_dict['dirt'])),
    'stat' : np.sqrt(np.diagonal(frac_cov_dict['stat_all'])), # does not include beam on STAT 
    'total' : np.sqrt(np.diagonal(tot_frac_cov))
}

In [ ]:
tot_unc = [0 for i in range(len(bins)-1)]
test = []

for source in frac_unc_dict.keys(): 
    
    if source=='total': 
        continue
    
    test.append(frac_unc_dict[source][0])
    
    # square the list 
    squared = [x**2 for x in frac_unc_dict[source]]
    
    # add in quadrature 
    tot_unc = [a+b for a,b in zip(tot_unc, squared)]
   
tot_unc = np.sqrt(np.array(tot_unc))
print(tot_unc)

frac_unc_dict['total'] = tot_unc

In [ ]:
frac_unc_dict['total']

In [ ]:


fig = plt.figure(figsize=(8, 5))  

# TOTAL 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Total",
        weights=frac_unc_dict['total'], linewidth=1.5, color='black')

# FLUX
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Flux", 
         weights=frac_unc_dict['flux'], color='royalblue')

# CROSS SECTION MODELS 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GENIE", 
         weights=frac_unc_dict['genie'], color='goldenrod')

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="GEANT4", 
         weights=frac_unc_dict['geant4'], color='green')

# DETECTOR 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Detector", 
         weights=frac_unc_dict['detector'], color='crimson')

# POT COUNTING 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="POT counting",
        weights=frac_unc_dict['pot_counting'], color='purple')

# DIRT 
plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Dirt",
        weights=frac_unc_dict['dirt'], color='brown')

# STATISTICAL 

plt.hist(bincenters, bins, histtype='step', range=[bins[0], bins[-1]], label="Stat (MC+EXT)",
        weights=frac_unc_dict['stat'], color='hotpink')

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.xlabel("Reco " + x_label, fontsize=15)
plt.ylabel("Fractional Uncertainty", fontsize=15)

plt.xlim(bins[0], xhigh)
plt.ylim(0, 0.5)

plt.legend(fontsize=13, frameon=False, ncol=3)
plt.title("FHC+RHC Background-Subtracted Event Rate", fontsize=16)

#plt.savefig(plots_path+xvar+"_RHC_FracUncertainty.pdf", transparent=True, bbox_inches='tight')

plt.show()

## Smearing & efficiency 

In [ ]:
with open('unfolding/smearing/FHC_'+xvar+'_July19.json') as f_fhc:
    fhc_smearing_dict = json.load(f_fhc)

In [ ]:
with open('unfolding/smearing/RHC_'+xvar+'_July19.json') as f_rhc:
    rhc_smearing_dict = json.load(f_rhc)

In [ ]:
###########################################################################################

# fhc efficiency 
fhc_true_generated = fhc_smearing_dict['true_generated']
fhc_true_selected = fhc_smearing_dict['true_selected']

fhc_eff = [a/b for a,b in zip(fhc_true_selected, fhc_true_generated)]
fhc_eff_err = [ np.sqrt( (x*(1-x)) / y ) for x,y in zip(fhc_eff,fhc_true_generated) ]

print('FHC Efficiency (%) = ', [round(x,3)*100 for x in fhc_eff])

###########################################################################################

# rhc efficiency 
rhc_true_generated = rhc_smearing_dict['true_generated']
rhc_true_selected = rhc_smearing_dict['true_selected']

rhc_eff = [a/b for a,b in zip(rhc_true_selected, rhc_true_generated)]
rhc_eff_err = [ np.sqrt( (x*(1-x)) / y ) for x,y in zip(rhc_eff,rhc_true_generated) ]

print('RHC Efficiency (%) = ', [round(x,3)*100 for x in rhc_eff])

###########################################################################################

# combined efficiency 

true_generated = [a+b for a,b in zip(fhc_smearing_dict['true_generated'], rhc_smearing_dict['true_generated'])]
true_selected = [a+b for a,b in zip(fhc_smearing_dict['true_selected'], rhc_smearing_dict['true_selected'])]

eff = [a/b for a,b in zip(true_selected, true_generated)]
eff_err = [ np.sqrt( (x*(1-x)) / y ) for x,y in zip(eff,true_generated) ]

print('Combined Efficiency (%) = ', [round(x,3)*100 for x in eff])


# Plot 

fig = plt.figure(figsize=(8, 5))  

plt.errorbar(bincenters, fhc_eff, yerr=fhc_eff_err, xerr=binwidth, label="FHC", fmt='.')
plt.errorbar(bincenters, rhc_eff, yerr=rhc_eff_err, xerr=binwidth, label="RHC", fmt='.')

plt.errorbar(bincenters, eff, yerr=eff_err, xerr=binwidth, label="FHC+RHC", fmt='.', color='black')

plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

plt.ylim(0, .25)

plt.xlabel("True " + x_label, fontsize=15)
plt.ylabel("Selected / Generated", fontsize=15)

plt.xlim(bins[0], xhigh)

plt.legend(fontsize=14, frameon=False, ncol=3)
plt.title("Efficiency", fontsize=16)

plt.grid(linestyle=':')

plt.show()

In [ ]:
response_matrix = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
smearing_matrix = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

fhc_response_matrix = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]
rhc_response_matrix = [ [0]*(len(bins)-1) for x in range(len(bins)-1) ]

for col in range(len(bins)-1):
    for row in range(len(bins)-1):

        smearing_matrix[row][col] = (fhc_smearing_dict['smearing_matrix'][row][col] + rhc_smearing_dict['smearing_matrix'][row][col])
        response_matrix[row][col] =  smearing_matrix[row][col] / true_generated[col]
        
        fhc_response_matrix[row][col] = fhc_smearing_dict['smearing_matrix'][row][col] / fhc_smearing_dict['true_generated'][col]
        rhc_response_matrix[row][col] = rhc_smearing_dict['smearing_matrix'][row][col] / rhc_smearing_dict['true_generated'][col]


In [ ]:
fig = plt.figure(figsize=(13, 9))

plt.pcolor(bins, bins, smearing_matrix, cmap='OrRd', vmin=0.00000000001, edgecolors='k')

plt.title('FHC+RHC Smearing', fontsize=15)

plt.xticks(x_ticks, fontsize=14)
plt.gca().xaxis.tick_bottom()
plt.yticks(x_ticks, fontsize=14)

plt.xlim(xlow, xhigh)
plt.ylim(xlow, xhigh)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

plt.xlabel('True '+x_label, fontsize=15)
plt.ylabel('Reco '+x_label, fontsize=15)

plt.show()

In [ ]:
fig = plt.figure(figsize=(13, 9))

plt.pcolor(bins, bins, response_matrix, cmap='OrRd', vmin=0.00000000001, edgecolors='k')

plt.title('FHC+RHC Smearing & Efficiency', fontsize=15)

plt.xticks(x_ticks, fontsize=14)
plt.gca().xaxis.tick_bottom()
plt.yticks(x_ticks, fontsize=14)

plt.xlim(xlow, xhigh)
plt.ylim(xlow, xhigh)
            
cbar = plt.colorbar()
cbar.ax.tick_params(labelsize=14)

plt.xlabel('True '+x_label, fontsize=15)
plt.ylabel('Reco '+x_label, fontsize=15)

plt.show()

In [ ]:
# check that this is constructed properly 
# sum of the columns should equal the combined efficiency 

for i in range(len(bincenters)): # loop over columns

    x = 0 
    for j in range(len(bincenters)): # sum up the rows
    
        x += response_matrix[j][i]

    print(x)

## CV event rate

In [ ]:
fig = plt.figure(figsize=(10, 7))


plt.hist(bincenters, bins, color='orange', label='True (Selected)', alpha=0.85,  histtype='step', linewidth=2, 
         weights=[a+b for a,b in zip(fhc_smearing_dict['true_selected'],rhc_smearing_dict['true_selected'])])#, linestyle=(0, (1, 1)))

plt.hist(bincenters, bins, color='blue', label='Reco (Selected)', linewidth=2, alpha=0.5, weights=evt_rate,histtype='step' )#,linestyle=(0, (1, 1)))

plt.legend(fontsize=13)
plt.title('FHC+RHC Selected Event Rate', fontsize=16)

plt.xlim(xlow, xhigh)

plt.xlabel(x_label, fontsize=15)
    
plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

#plt.savefig(plots_path+"_InputSignalDistributions_ElectronEnergy.pdf", transparent=True, bbox_inches='tight') 

#plt.grid()
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 7))


plt.hist(bincenters, bins, color='orange', label='True (Generated)', alpha=0.4, 
         weights=[a+b for a,b in zip(fhc_smearing_dict['true_generated'],rhc_smearing_dict['true_generated'])])

plt.hist(bincenters, bins, color='blue', label='Reco (Selected)', alpha=0.4, weights=evt_rate)

plt.legend(fontsize=13)
plt.title('FHC+RHC Input Signal Distributions', fontsize=16)

plt.xlim(xlow, xhigh)

plt.xlabel(x_label, fontsize=15)
    
plt.xticks(x_ticks, fontsize=13)
plt.yticks(fontsize=13)

#plt.savefig(plots_path+"_InputSignalDistributions_ElectronEnergy.pdf", transparent=True, bbox_inches='tight') 

plt.grid()
plt.show()

## save to unfolding file 

In [ ]:
save = True

if save: 
    print("Make sure to update file name!")
    path = "/uboone/data/users/kmiller/uBNuMI_CCNp/unfolding/input/"
    

In [ ]:
# covariance matrix 
fhc_hcov = TH2D("hcov_tot", "Covariance Matrix vs. Reco "+x_label, 
                len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))

for i in range(len(bincenters)): # i = row (y)
    for j in range(len(bincenters)): # j = column (x) 

        fhc_hcov.Fill(bincenters[j], bincenters[i], fhc_tot_abs_cov[i][j]) 

# true signal distribution
fhc_htrue_signal = TH1F("htrue_signal", "Generated MC Signal vs. True "+x_label+" ("+true_var+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    fhc_htrue_signal.Fill(bincenters[i], fhc_smearing_dict['true_generated'][i])
        
# reco distribution (MC or data)
fhc_hreco = TH1F("hmeas", "Selected MC Signal vs. Reco "+x_label+" ("+xvar+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    fhc_hreco.Fill(bincenters[i], fhc_dict['evt_rate'][i])
    
# response matrix 
fhc_r = TH2D("hR", "Response Matrix", len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))
for i in range(len(bincenters)): # i = column (x)
    for j in range(len(bincenters)): # j = row (y) 
            
        fhc_r.Fill(bincenters[i], bincenters[j], fhc_response_matrix[i][j]) 
            

    

In [ ]:
fhc_filename = "WSVD_"+xvar+"_FHC_"+date.today().strftime("%m%d%y")+".root"
    
if os.path.exists(path+fhc_filename): 
    print(fhc_filename, "exists and is readable, need to update file name to save ! ")
        
else: 
            
    fhc_f = ROOT.TFile.Open(path+fhc_filename, "RECREATE")

    fhc_f.cd()

    fhc_hcov.Write()
    fhc_htrue_signal.Write()
    fhc_hreco.Write()
    fhc_r.Write()

    fhc_f.Close()

In [ ]:
 # covariance matrix 
rhc_hcov = TH2D("hcov_tot", "Covariance Matrix vs. Reco "+x_label, 
            len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))

for i in range(len(bincenters)): # i = row (y)
    for j in range(len(bincenters)): # j = column (x) 

        rhc_hcov.Fill(bincenters[j], bincenters[i], rhc_tot_abs_cov[i][j]) 

# true signal distribution
rhc_htrue_signal = TH1F("htrue_signal", "Generated MC Signal vs. True "+x_label+" ("+true_var+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    rhc_htrue_signal.Fill(bincenters[i], rhc_smearing_dict['true_generated'][i])
        
# reco distribution (MC or data)
rhc_hreco = TH1D("hmeas", "Selected MC Signal vs. Reco "+x_label+" ("+xvar+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    rhc_hreco.Fill(bincenters[i], rhc_dict['evt_rate'][i])
    
# response matrix 
rhc_r = TH2D("hR", "Response Matrix", len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))
for i in range(len(bincenters)): # i = column (x)
    for j in range(len(bincenters)): # j = row (y) 
            
        rhc_r.Fill(bincenters[i], bincenters[j], rhc_response_matrix[i][j]) 
            

rhc_filename = "WSVD_"+xvar+"_RHC_"+date.today().strftime("%m%d%y")+".root"
    
if os.path.exists(path+rhc_filename): 
    print(rhc_filename, " exists and is readable, need to update file name to save ! ")
        
else: 
            
    rhc_f = ROOT.TFile.Open(path+rhc_filename, "RECREATE")

    rhc_f.cd()
    rhc_hcov.Write()
    rhc_htrue_signal.Write()
    rhc_hreco.Write()
    rhc_r.Write()

    rhc_f.Close()



In [ ]:
# covariance matrix 
hcov = TH2D("hcov_tot", "Covariance Matrix vs. Reco "+x_label, 
            len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))

for i in range(len(bincenters)): # i = row (y)
    for j in range(len(bincenters)): # j = column (x) 

        hcov.Fill(bincenters[j], bincenters[i], tot_abs_cov[i][j]) 

# true signal distribution
htrue_signal = TH1F("htrue_signal", "Generated MC Signal vs. True "+x_label+" ("+true_var+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    htrue_signal.Fill(bincenters[i], true_generated[i])
        
# reco distribution (MC or data)
hreco = TH1D("hmeas", "Selected MC Signal vs. Reco "+x_label+" ("+xvar+")", len(bins)-1, np.array(bins))

for i in range(len(bincenters)): 
    hreco.Fill(bincenters[i], evt_rate[i])
    
# response matrix 
r = TH2D("hR", "Response Matrix", len(bins)-1, np.array(bins), len(bins)-1, np.array(bins))
for i in range(len(bincenters)): # i = column (x)
    for j in range(len(bincenters)): # j = row (y) 
            
        r.Fill(bincenters[i], bincenters[j], response_matrix[i][j]) 
            
            
filename = "WSVD_"+xvar+"_Combined_"+date.today().strftime("%m%d%y")+".root"
    
if os.path.exists(path+filename): 
    print(filename, " exists and is readable, need to update file name to save ! ")
        
else: 
            
    f = ROOT.TFile.Open(path+filename, "RECREATE")

    f.cd()

    hcov.Write()
    htrue_signal.Write()
    hreco.Write()
    r.Write()

    f.Close()            
